# Contexte

Market Basket Analysis(Wordcloud):

In the retail sector, this analysis is used to discover patterns and relationships within transactions and transaction items-sets (Items frequently bought together).

IBM puts it as:
“Market Basket Analysis is used to increase marketing effectiveness and to improve cross-sell and up-sell opportunities by making the right offer to the right customer. For a retailer, good promotions translate into increased revenue and profits. The objectives of the market basket analysis models are to identify the next product that the customer might be interested to purchase or to browse.”

This explanation gives the intent of running the analysis: “to improve cross-sell and up-sell opportunities.

# Import

In [14]:
import warnings
warnings.filterwarnings('ignore')

import pickle
from IPython.display import clear_output

In [15]:
# First we need to install the following modules to get apriori algorithm and the word cloud outputs.Ignore if installed.
#!pip install mlxtend(# For directly installing in jupyter nb)
#!pip install wordcloud(# For directly installing in jupyter nb)
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori,association_rules
# Loading the data
data=pd.read_csv("base_sql_basket.csv")
# glimpse of the dataset
data.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France
1,1,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583.0,France
2,2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France
3,3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583.0,France
4,4,536370,21883,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583.0,France


In [16]:
# Columns of the dataset
data.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'Description', 'Quantity',
       'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [17]:
# Shape of the data 
data.shape

(44553, 9)

In [18]:
# Information of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   44553 non-null  int64  
 1   InvoiceNo    44553 non-null  int64  
 2   StockCode    44553 non-null  object 
 3   Description  44553 non-null  object 
 4   Quantity     44553 non-null  int64  
 5   InvoiceDate  44553 non-null  object 
 6   UnitPrice    44553 non-null  float64
 7   CustomerID   43335 non-null  float64
 8   Country      44553 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 3.1+ MB


In [19]:
# Missing values count
data.isna().sum()

Unnamed: 0        0
InvoiceNo         0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID     1218
Country           0
dtype: int64

In [20]:
# Exploring the different unique countries of transactions
data.Country.unique()

array(['France', 'Australia', 'Netherlands', 'Germany', 'Norway', 'EIRE',
       'Switzerland', 'Spain', 'Poland', 'Portugal', 'Italy', 'Belgium',
       'Lithuania', 'Japan', 'Iceland', 'Channel Islands', 'Denmark',
       'Cyprus', 'Sweden', 'Finland', 'Austria', 'Bahrain', 'Israel',
       'Greece', 'Hong Kong', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Saudi Arabia', 'Czech Republic', 'Canada',
       'Brazil', 'USA', 'European Community', 'Malta', 'RSA'],
      dtype=object)

In [21]:
# Checking different number of unique countries present in this dataset
x = data['Country'].nunique()
print("There are {} number of unique countries.".format(x))

There are 36 number of unique countries.


In [22]:
# Total number of unique transactions
a=len(data['InvoiceNo'].unique())
print("There are a total of {} unique transactions.".format(a))

There are a total of 1929 unique transactions.


In [23]:
# checking how many unique customer IDs are there
b = data['CustomerID'].nunique()
print("There are {} number of different customers.".format(b))

There are 414 number of different customers.


# Cleaning the data

In [24]:
# Removing the extra spaces in the description column using strip()
data['Description'] = data['Description'].str.strip()
# Changing the type of invoice_no to string to help in removing the transactions cancelled/done on credit
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
# Dropping all transactions which were cancelled/done on credit 
data = data[~data['InvoiceNo'].str.contains('C')]
data = data.drop(columns= "Unnamed: 0")
# Checking the shape of the data after removing the transactions which were cancelled
data.shape

(44553, 8)

# One-hot Encoding

In [25]:
# Defining the hot encoding function to make the data suitable for the concerned libraries 
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

# Preparing the data for modelisation

In [26]:
good_countries =["France", "Netherlands", "Germany", "EIRE", "Spain"
                ,"Portugal", "Belgium", "Sweden", "Finland", "Bahrain", "Saudi Arabia"]

In [27]:
#basket per country
basket_pays = {}
for i in good_countries:
    basket_country = (data[data['Country'] == f'{i}']
              .groupby(['InvoiceNo', 'Description'])['Quantity']
              .sum().unstack().reset_index().fillna(0)
              .set_index('InvoiceNo'))
    basket_encoded = basket_country.applymap(hot_encode)
    basket_pays[i] = basket_encoded

# Building the models and analyzing the results:

In [28]:
basket_rules = {}
for i in good_countries:
    frq_items = apriori(basket_pays.get(i), min_support = 0.05, use_colnames = True) 

    # Collecting the inferred rules in a dataframe 
    rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
    rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
    basket_rules[i] = rules

In [29]:
basket_rules.keys()

dict_keys(['France', 'Netherlands', 'Germany', 'EIRE', 'Spain', 'Portugal', 'Belgium', 'Sweden', 'Finland', 'Bahrain', 'Saudi Arabia'])

In [30]:
pickle.dump(basket_rules, open("basket_rules.pkl", "wb"))

In [31]:
basket_rules.get("France").head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
45,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.076531,0.765306,0.076531,1.000000,1.306667,0.017961,inf,0.254144
258,"(PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...",(POSTAGE),0.051020,0.765306,0.051020,1.000000,1.306667,0.011974,inf,0.247312
271,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...",(POSTAGE),0.053571,0.765306,0.053571,1.000000,1.306667,0.012573,inf,0.247978
300,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959,0.967949
301,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796,0.956294
334,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.084184,0.127551,0.081633,0.969697,7.602424,0.070895,28.790816,0.948294
336,"(SET/20 RED RETROSPOT PAPER NAPKINS, POSTAGE, ...",(SET/6 RED SPOTTY PAPER CUPS),0.084184,0.137755,0.081633,0.969697,7.039282,0.070036,28.454082,0.936804
128,(RED RETROSPOT PICNIC BAG),(POSTAGE),0.071429,0.765306,0.068878,0.964286,1.260000,0.014213,6.571429,0.222222
143,(SET OF 9 BLACK SKULL BALLOONS),(POSTAGE),0.066327,0.765306,0.063776,0.961538,1.256410,0.013015,6.102041,0.218579
175,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122,0.981725
